In [4]:
%load_ext autoreload
import sys
sys.path.append("..")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
%matplotlib inline

In [6]:
%autoreload

import matplotlib
matplotlib.rc_file('matplotlibrc')
import matplotlib.pyplot as plt

import numpy as np

import darkhistory.physics as phys

import main_dm_baryon_scattering as main

from tqdm import tqdm_notebook as tqdm

import pickle

In [8]:
import config
from scipy.interpolate import interp1d
from scipy.interpolate import interp2d

decay_elec_CMB_raw = np.loadtxt(config.data_path+'/CMB_limits_elec_decay.csv', delimiter=',')
decay_phot_CMB_raw = np.loadtxt(config.data_path+'/CMB_limits_phot_decay.csv', delimiter=',')

decay_elec_CMB = interp1d(np.transpose(decay_elec_CMB_raw)[0,:], np.transpose(decay_elec_CMB_raw)[1,:])
decay_phot_CMB = interp1d(np.transpose(decay_phot_CMB_raw)[0,:], np.transpose(decay_phot_CMB_raw)[1,:])

def xsec_bound_elec_CMB(mDM, DM_process):
    if DM_process == 'swave':
        return 4.1e-28*(mDM*1e-9)/f_elec_CMB(np.log10(mDM-phys.me), np.log10(601))[0]
    elif DM_process == 'decay':
        return np.array([decay_elec_CMB(mDM*1e-9)])[0]

def xsec_bound_phot_CMB(mDM, DM_process):
    if DM_process == 'swave':
        return 4.1e-28*(mDM*1e-9)/f_phot_CMB(np.log10(mDM), np.log10(601))[0]
    elif DM_process == 'decay':
        return np.array([decay_phot_CMB(mDM*1e-9)])[0]

OSError: /Users/gridgway/Downloads/dataverse_files/CMB_limits_elec_decay.csv not found.

In [5]:
struct_boost = phys.struct_boost_func(model='einasto_subs')
struct_boost_NFW = phys.struct_boost_func(model='NFW_no_subs')
struct_boost_pwave = phys.struct_boost_func(model='pwave_NFW_no_subs')

# Improved f_exc

In [ ]:
mDM=1e6
lifetime=3e25
sigmav=3e-26
datums = main.evolve(
    DM_process='decay', mDM=mDM, primary='phot_delta',
    start_rs = 3000, sigmav=sigmav, lifetime=lifetime,
    coarsen_factor=4, backreaction=False,
    struct_boost=struct_boost_pwave,
    end_rs=4.01
)

In [ ]:
import darkhistory.low_energy.lowE_electrons as lowE_elec
# import darkhistory.low_energy.lowE_photons as lowE_phot
# import darkhistory.low_energy.lowE_deposition as lowE
MEDEA_interp = lowE_elec.make_interpolator()

In [ ]:
# elec_exc = [lowE_elec.compute_fs(MEDEA_interp, 
#            datums['lowengelec'][i], 
#            datums['x'][i,0], 
#            phys.inj_rate('pwave', rs_list[i], mDM=1e8, sigmav=3e-24), 
#            4*.001/phys.hubble(rs_list[i]))[2]
#  for i in np.arange(len(rs_list))]

phot_exc = [lowE_phot.compute_fs( 
           datums['lowengphot'][i],
           np.array([
                1. - phys.xHII_std(rs_list[i]),
                phys.chi - phys.xHeII_std(rs_list[i]),
                phys.xHeII_std(rs_list[i])
            ]), 
           phys.inj_rate('decay', rs_list[i], mDM=mDM, sigmav=sigmav, lifetime=lifetime), 
           4*.001/phys.hubble(rs_list[i]), method='old')[2]
 for i in np.arange(len(rs_list))]

# tot_exc = [lowE.compute_fs(MEDEA_interp, 
#            datums['lowengelec'][i], 
#            datums['lowengphot'][i], 
#            np.array([
#                 1. - phys.xHII_std(rs_list[i]),
#                 phys.chi - phys.xHeII_std(rs_list[i]),
#                 phys.xHeII_std(rs_list[i])
#             ]), 
#            phys.inj_rate('pwave', rs_list[i], mDM=1e9, sigmav=3e-24), 
#            4*.001/phys.hubble(rs_list[i]), 
#            [0,0,0,0,0], cmbloss=0, 
#            method='no_He', 
#            separate_higheng=True, 
#            cross_check=False)[0][2]
# for i in np.arange(len(rs_list))]

# tot_exc = [lowE.compute_fs(MEDEA_interp, 
#            datums['lowengelec'][i], 
#            datums['x'][i,0], 
#            phys.inj_rate('pwave', rs_list[i], mDM=1e8, sigmav=3e-24), 
#            4*.001/phys.hubble(rs_list[i]))[2]
#  for i in np.arange(len(rs_list))]

In [ ]:
fexc = datums['f']['low']['exc']+0*datums['f']['high']['exc']
rs_list = datums['lowengphot'].rs
nrms = phys.nB * rs_list**3 / (4*.001/phys.hubble(rs_list)) /(
    phys.inj_rate('decay', rs_list, mDM=mDM, sigmav=sigmav, lifetime=lifetime)
)
fexc2 = datums['lowengphot'].toteng(bound_type='eng',
                            bound_arr=np.array([phys.lya_eng,phys.rydberg])
                           )*nrms

In [ ]:
plt.figure()
ax = plt.gca()
plt.axis([4,3e3,1e-4,5])
ax.loglog()
plt.plot(rs_list,fexc)
plt.plot(rs_list,fexc2[0])
# plt.plot(rs_list,phys.f_std(1e9,rs_list,'phot','pwave',True,'exc'))
# plt.plot(rs_list,fexc2[0]+elec_exc)
# plt.plot(rs_list,tot_exc)
plt.plot(rs_list,phot_exc)

In [ ]:
plt.figure()
ax = plt.gca()
plt.axis([4,3e3,1e-6,5])
ax.loglog()
plt.plot(rs_list,fexc)
plt.plot(rs_list,fexc2[0])
# plt.plot(rs_list,phys.f_std(1e9,rs_list,'phot','pwave',True,'exc'))
# plt.plot(rs_list,fexc2[0]+elec_exc)
# plt.plot(rs_list,tot_exc)
plt.plot(rs_list,phot_exc)